PEFT documentation used for Efficient Fine Tuning https://huggingface.co/docs/peft/index
Accelerate Documentation- To adapt to the available config https://huggingface.co/docs/accelerate/index

Transformers

Teaching French to LLama-1-7b model with Fine tuning

In [ ]:
!pip install transformers
!pip install accelerate
!pip install peft

PEFT documentation used for Efficient Fine Tuning https://huggingface.co/docs/peft/index
Accelerate Documentation- To adapt to the available config https://huggingface.co/docs/accelerate/index

Transformers

Trl for supervised fine tuning (sft) https://huggingface.co/docs/trl/main/en/sft_trainer

Datasets to load hf datasets

einops is for the falcon model requirement



In [ ]:
!pip install datasets
!pip install bitsandbytes
!pip install trl
!pip install einops
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 649.0 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/410.6 MB 858.8 kB/s eta 0:06:19
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 1.8 MB/s eta 0:00:00
  Using cached accelerate-0.29.0-py3-no

https://huggingface.co/transformers/v3.0.2/model_doc/auto.html

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('TinyPixel/Llama-2-7B-bf16-sharded')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

QLoRA https://huggingface.co/blog/4bit-transformers-bitsandbytes

In [ ]:
import torch
from transformers import BitsAndBytesConfig


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

PackageNotFoundError: No package metadata was found for bitsandbytes

Casual Modelling

need to either downgrade transformers to 4.30 or change runtime to gpu

But quantizaiton needs CUDA 11.2 on which it runs accelerate and the bitsandbytes, so upgrading the runtime to gpu

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
model = AutoModelForCausalLM.from_pretrained('TinyPixel/Llama-2-7B-bf16-sharded', quantization_config = nf4_config, device_map="auto")

setting the config for qlora

In [ ]:
from peft import LoraConfig,PeftConfig,get_peft_model,prepare_model_for_kbit_training
Lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

Can add target modules for peft

Setting the trainable arguments for finetuning with peft
https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir='./output',
    auto_find_batch_size=True,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=40,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=40,
    warmup_ratio=0.02,
    group_by_length=True,
    lr_scheduler_type="constant",
    logging_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=4)

Using SFT trainer that gives a wrapper around the trainer class of transformer. This is the core of RLHF https://huggingface.co/docs/trl/main/en/sft_trainer

Fine tuning a fully quantized model is not possible. Need to add adapters on top of the trained models with something like peft https://github.com/huggingface/transformers/issues/27702



In [ ]:
from datasets import Dataset

# Define your dialogue dataset
dialogue_dataset = [
    {"human": "What is Smart Energy in the Informatic Age?", "assistant": "It's the use of information technology to optimize energy use, integrating devices and systems in homes for efficiency and convenience."},
    {"human": "How do Smart Homes contribute to energy efficiency?", "assistant": "By using devices like smart thermostats and energy management systems to automate tasks and provide data for informed energy use decisions."},
    {"human": "What are some examples of Smart Home devices?", "assistant": "Devices include smart thermostats, lighting systems, energy-efficient appliances, and solar energy systems, all enhancing efficiency and control."},
    {"human": "Can Smart Homes be powered by renewable energy?", "assistant": "Yes, they can integrate with sources like solar panels, using smart systems to manage and maximize the use of clean energy."},
    {"human": "What is the role of AI in Smart Energy Systems?", "assistant": "AI analyzes data to optimize energy consumption, predicting usage patterns and automating systems for greater efficiency."},
    {"human": "How secure are Smart Home Energy Systems?", "assistant": "Security measures include encryption and regular updates. Users should also maintain strong passwords and secure network settings."},
    {"human": "What are the environmental benefits of Smart Homes?", "assistant": "They reduce energy consumption and carbon emissions, promoting sustainable living by optimizing resource use and integrating renewable energy."},
    {"human": "Are Smart Home Energy Systems expensive to install?", "assistant": "Costs vary, but long-term savings and potential incentives can offset initial expenses, making them a valuable investment."},
    {"human": "Can I retrofit my existing home with Smart Energy Systems?", "assistant": "Many smart devices can be easily added to existing homes, while more extensive systems might require professional installation."},
    {"human": "How can I get started with making my home smarter and more energy-efficient?", "assistant": "Begin by assessing your energy use, installing simple devices like smart thermostats, and considering a consultation for comprehensive upgrades."},
    {"human": "What is the importance of interoperability in Smart Homes?", "assistant": "It ensures that different devices can work together seamlessly, creating an integrated and user-friendly system."},
    {"human": "How do Smart Meters contribute to energy efficiency in Smart Homes?", "assistant": "They provide real-time energy use data, helping monitor and optimize consumption for efficiency."},
    {"human": "Can Smart Home systems predict and adapt to my energy usage patterns?", "assistant": "Yes, with learning algorithms, they automate tasks based on your habits, optimizing energy use without compromising comfort."},
    {"human": "What role does data privacy play in Smart Homes?", "assistant": "Protecting personal data is crucial, with manufacturers adhering to strict regulations and offering transparent policies."},
    {"human": "How can I ensure that my Smart Home devices remain up-to-date and secure?", "assistant": "Regularly update device software, use strong passwords, and maintain a secure network to enhance device security and functionality."}
]

# Convert dialogue dataset to the desired format
formatted_dataset = []
for idx, example in enumerate(dialogue_dataset, start=1):
    formatted_example = f"<s>[INST] Human: {example['human']} Assistant: {example['assistant']} [/INST]</s>"
    formatted_dataset.append(formatted_example)

# Create a Hugging Face Dataset object
dataset = Dataset.from_dict({"text": formatted_dataset})

In [ ]:
print(dataset)

In [ ]:
dataset['text'][0]


Use formatting prompts and DataCollator here

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_args,
    peft_config=Lora_config,
    max_seq_length = 512

)


Optimization

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
!pip install wandb



In [ ]:
trainer.train()

In [ ]:
text = "Human: What is Smart Energy in the Informatic Age?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=70)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:

  text = "Human: How do Smart Meters contribute to energy efficiency in Smart Homes?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_old.generate(**inputs, max_new_tokens=70)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: What is Smart Energy in the Informatic Age? Assistant: It's the use of information technology to optimize energy use, integrating devices and systems in homes for efficiency and convenience. [/]</s></s> </ajax> </s> </span> </div> </span> </li> </ul> </span> </div> </span> </li>


In [ ]:
text = "Human: How do Smart Meters contribute to energy efficiency in Smart Homes?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=70)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: How do Smart Meters contribute to energy efficiency in Smart Homes? Assistant: They provide real-time energy use data, helping monitor and optimize consumption for efficiency. [/]</s></s></span></span></label> <input type="text" class="form-control" value="They provide real-time energy use data, helping monitor and optimize consumption for efficiency.">


Saving only the adapters instead of the whole model
The SFT Trainer gives methods to only store the adapters

In [ ]:
saving_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
saving_model.save_pretrained("trained_output")

In [ ]:
aquired_lora_config = LoraConfig.from_pretrained("trained_output")
peft_model = get_peft_model(model,aquired_lora_config)

Pushing to hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub("llama2-fine-tuned)